In [1]:
import pandas as pd
import duckdb
import os

In [2]:
sensor = pd.read_csv("../Data/NAV/sensor.csv")
sensorbox = pd.read_csv("../Data/NAV/sensorbox.csv")
home = pd.read_csv("../Data/NAV/home.csv")

In [3]:
sensorbox["sensorbox_type"].unique()

array(['room', 'clamp', 'electric', 'gas', 'relay', 'plug_monitor',
       'subcircuit_monitor'], dtype=object)

In [4]:
sensor["type"].unique()

array(['light', 'humidity', 'temperature', 'battery', 'clamp_temperature',
       'electric', 'gas', 'electric_socket', 'power', 'electric_pulse'],
      dtype=object)

In [5]:
duckdb.query("""
    select a.*
    from '../Data/NAV/sensor.csv' as a
    """).to_df()

,sensorid,sensorboxid,type,unit,status,roomid,subcircuit_type,scalingfactor,rawunit,counter
0,1174,279492261292874,light,0.1cd,active,650,None,1.0,None,1
1,1175,279492261292874,humidity,0.1%,active,650,None,1.0,None,1
2,1176,279492261292874,temperature,0.1C,active,650,None,1.0,None,1
3,1177,279492261292874,battery,0.01V,active,650,None,1.0,None,1
4,1178,279492261292874,battery,0.01V,active,650,None,1.0,None,2
...,...,...,...,...,...,...,...,...,...,...
20076,21843,279492261297010,humidity,0.1%,active,3026,None,1.0,None,1
20077,21844,279492261297010,temperature,0.1C,active,3026,None,1.0,None,1
20078,21845,279492261297010,battery,0.01V,active,3026,None,1.0,None,1
20079,21846,279492261297010,battery,0.01V,active,3026,None,1.0,None,2


### Wypisanie nazw plików dla wybranego domu

In [6]:
def returm_matching_files(folder_path, searched_fragment):
    matching_files = [
        filename
        for filename in os.listdir(folder_path)
        if searched_fragment in filename and not filename.startswith("._")
    ]
    return pd.DataFrame(matching_files, columns=['file_path'])

In [7]:
folder_path = "../Data/CSV/household_sensors/sensordata"
wanted_fragment = "home61"

household_61 = returm_matching_files(folder_path, wanted_fragment)

In [8]:
folder_path = "../Data/CSV/room_and_appliance_sensors/sensordata"
wanted_fragment = "home61"

room_and_appliance_61 = returm_matching_files(folder_path, wanted_fragment)

### Sprawdzenie który dom ma najwięcej plików i przejrzenie ich

In [9]:
def policz_prefixy(folder_path):
    """
    Zwraca DataFrame z unikalnymi prefixami (do pierwszego podkreślenia) 
    z nazw plików w folderze i liczbą ich wystąpień.
    
    Args:
        folder_path (str): Ścieżka do folderu z plikami.
        
    Returns:
        pd.DataFrame: Kolumny: 'prefix', 'count'
    """
    # Pobierz wszystkie zwykłe pliki (pomijając ukryte i pliki typu ._)
    filenames = [
        f for f in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, f)) and not f.startswith("._")
    ]

    # Wyciągnij prefiks do pierwszego "_"
    prefixes = [fname.split("_")[0] for fname in filenames]

    # Policz wystąpienia i stwórz DataFrame
    df_counts = pd.Series(prefixes).value_counts().reset_index()
    df_counts.columns = ["prefix", "count"]

    return df_counts

In [10]:
folder_path = "../Data/CSV/household_sensors/sensordata"
household_counts = policz_prefixy(folder_path)

folder_path = "../Data/CSV/room_and_appliance_sensors/sensordata"
room_and_appliance_counts = policz_prefixy(folder_path)

In [11]:
household_counts.sort_values(by="count", ascending=False, inplace=True)
room_and_appliance_counts.sort_values(by="count", ascending=False, inplace=True)

In [12]:
# Zmiana nazwy kolumn
household_counts = household_counts.rename(columns={'count': 'household_count'})
room_and_appliance_counts = room_and_appliance_counts.rename(columns={'count': 'room_appliance_count'})

# Połączenie po prefixie
merged = pd.merge(household_counts, room_and_appliance_counts, on='prefix', how='inner')

# DDodanie kolumny z sumą liczb wystąpień
merged['total_count'] = merged['household_count'] + merged['room_appliance_count']

# Sortowanie i wybranie najczęściej występującego prefixu
top_comon = merged.sort_values('total_count', ascending=False).head(1)

print(top_common)

NameError: name 'top_common' is not defined

### Przejrzenie plików dla home146

In [13]:
folder_path_household = "../Data/CSV/household_sensors/sensordata"
folder_path_room_and_appliance = "../Data/CSV/room_and_appliance_sensors/sensordata"

wanted_fragment_household = "home146"
wanted_fragment_room_and_appliance = "home146"

household_146 = returm_matching_files(folder_path_household, wanted_fragment_household)
room_and_appliance_146 = returm_matching_files(folder_path_room_and_appliance, wanted_fragment_room_and_appliance)

In [14]:
room_and_appliance_146

,file_path
0,home146_bathroom1357_sensor15382_tempprobe_rad...
1,home146_bathroom1357_sensor15383_tempprobe_rad...
2,home146_bathroom1357_sensor15466_tempprobe_sin...
3,home146_bathroom1357_sensor7176_room_humidity....
4,home146_bathroom1357_sensor7177_room_temperatu...
5,home146_bathroom1361_sensor15447_tempprobe_rad...
6,home146_bathroom1361_sensor15448_tempprobe_rad...
7,home146_bathroom1361_sensor15454_tempprobe_sin...
8,home146_bathroom1361_sensor7214_room_humidity....
9,home146_bathroom1361_sensor7215_room_temperatu...


In [15]:
household_146.to_excel("household_146.xlsx", index=False)
room_and_appliance_146.to_excel("room_and_appliance_146.xlsx", index=False)

### Sprawdzenie jakie są unikalne nazwy zawierające "electric" w folderze household_sensors

In [16]:
def find_unique_electric_segments(folder_path):
    """
    Returns unique parts of filenames that contain 'electric',
    specifically the segment between underscores that contains 'electric'.
    
    Args:
        folder_path (str): Path to the folder containing the files.
        
    Returns:
        set: A set of unique segments (str) that contain 'electric'.
    """
    unique_segments = set()
    
    for filename in os.listdir(folder_path):
        if "electric" in filename and not filename.startswith("._"):
            parts = filename.split("_")
            for part in parts:
                if "electric" in part:
                    unique_segments.add(part)
    
    return unique_segments

In [17]:
folder_path_household = "../Data/CSV/household_sensors/sensordata"

find_unique_electric_segments(folder_path_household)

{'electric-combined.csv.gz', 'electric-mains', 'electric-subcircuit'}

In [18]:
from collections import defaultdict

def count_electric_files_per_home(data_directory):
    """
    Counts the number of files containing the word 'electric' for each home,
    based on IDEAL dataset naming convention.

    Args:
        data_directory (str): Path to the directory containing the sensor data files.

    Returns:
        dict: A dictionary where keys are home IDs and values are counts of 'electric' files.
    """
    electric_file_counts = defaultdict(int)

    for filename in os.listdir(data_directory):
        if "electric" in filename and filename.endswith(".csv.gz"):
            if filename.startswith("home"):
                try:
                    home_id = filename.split("_")[0].replace("home", "")
                    electric_file_counts[home_id] += 1
                except IndexError:
                    continue  # Skip files with unexpected naming format

    return pd.DataFrame(sorted(electric_file_counts.items(), key=lambda item: int(item[0])))

In [19]:
electric_per_home = count_electric_files_per_home("../Data/CSV/household_sensors/sensordata")

In [20]:
electric_per_home.sort_values(by=1, ascending=False, inplace=True)

In [21]:
electric_per_home

,0,1
100,171,5
6,65,4
43,106,4
103,175,4
91,162,4
...,...,...
249,331,1
250,332,1
251,333,1
252,334,1
